In [1]:
import pandas as pd
import os
import numpy as np

### General dataframe

In [2]:
general_df = pd.read_csv("data/population_data/country_area.csv")

general_df = general_df[(general_df['Year'] <= 2023) & (general_df['Year'] >= 2000)]

# Drop redundant columns
general_df.drop(columns=['ISO2 Alpha-code', 'SDMX code**'], inplace=True)

# Rename columns
general_df.rename(columns={"Region, subregion, country or area *": "Country",
                           "Location code": "location_code",
                           "ISO3 Alpha-code": "country_code",
                           "Total Population, as of 1 July (thousands)": "total_population",
                           "Male Population, as of 1 July (thousands)": "male_population",
                           "Female Population, as of 1 July (thousands)": "female_population",
                           "Population Density, as of 1 July (persons per square km)": "population_density",
                           "Life Expectancy at Birth, both sexes (years)": "life_expectancy_at_birth",
                           "Male Life Expectancy at Birth (years)": "male_life_expectancy_at_birth",
                           "Female Life Expectancy at Birth (years)": "female_life_expectancy_at_birth",
                           "Infant Mortality Rate (infant deaths per 1,000 live births)": "infant_deaths_per_1000",
                           "Mortality between Age 15 and 60, both sexes (deaths under age 60 per 1,000 alive at age 15)": "mortality_between_15_60_per_1000"
                           }, inplace=True)
general_df

,Country,location_code,country_code,Type,Year,total_population,male_population,female_population,population_density,life_expectancy_at_birth,male_life_expectancy_at_birth,female_life_expectancy_at_birth,infant_deaths_per_1000,mortality_between_15_60_per_1000
50,Afghanistan,4,AFG,Country/Area,2000,19542.982,9815.442,9727.541,30.099,55.298,53.762,56.858,90.813,322.256
51,Afghanistan,4,AFG,Country/Area,2001,19688.632,9895.467,9793.166,30.323,55.798,54.272,57.344,88.358,316.523
52,Afghanistan,4,AFG,Country/Area,2002,21000.256,10562.202,10438.055,32.343,56.454,55.227,57.672,85.779,306.104
53,Afghanistan,4,AFG,Country/Area,2003,22645.130,11397.483,11247.647,34.876,57.344,56.032,58.650,82.600,295.454
54,Afghanistan,4,AFG,Country/Area,2004,23553.551,11862.726,11690.825,36.276,57.944,56.658,59.216,79.936,288.209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35705,Zimbabwe,716,ZWE,Country/Area,2019,15354.608,7231.989,8122.618,39.691,61.292,58.644,63.661,37.195,347.489
35706,Zimbabwe,716,ZWE,Country/Area,2020,15669.666,7385.220,8284.447,40.506,61.124,58.129,63.862,36.810,349.046
35707,Zimbabwe,716,ZWE,Country/Area,2021,15993.524,7543.690,8449.834,41.343,59.253,56.232,62.045,36.760,386.447
35708,Zimbabwe,716,ZWE,Country/Area,2022,16320.537,7705.601,8614.935,42.188,59.391,56.428,62.094,37.421,382.439


### Merge the economy dataframes

In [3]:
# Define the rename mapping
country_name_replace_mapping_for_economy_df = {"Bahamas, The": "Bahamas",
                                               "Congo, Republic of ": "Republic of the Congo", "Congo, Dem. Rep. of the": "Democratic Republic of the Congo",
                                               "Czech Republic": "Czechia",
                                               "Korea, Republic of": "South Korea",
                                               "Gambia, The": "The Gambia",
                                               "Hong Kong SAR": "Hong Kong",
                                               "China, People's Republic of": "China",
                                               "Kyrgyz Republic": "Kyrgyzstan",
                                               "Lao P.D.R.": "Laos",
                                               "North Macedonia ": "North Macedonia",
                                               "Slovak Republic": "Slovakia",
                                               "South Sudan, Republic of": "South Sudan",
                                               "Micronesia, Fed. States of": "Federated States of Micronesia",
                                               "Türkiye, Republic of": "Turkey",
                                               "Taiwan Province of China": "Taiwan",
                                               "Macao SAR": "Macao"
                                               }

country_name_replace_mapping_for_general_df = {"Bolivia (Plurinational State of)": "Bolivia",
                                               "Congo": "Republic of the Congo",
                                               "Republic of Korea": "South Korea",
                                               "Gambia": "The Gambia",
                                               "China, Hong Kong SAR": "Hong Kong",
                                               "Iran (Islamic Republic of)": "Iran",
                                               "Kosovo (under UNSC res. 1244)": "Kosovo",
                                               "Lao People's Democratic Republic": "Laos",
                                               "Syrian Arab Republic": "Syria",
                                               "Micronesia (Fed. States of)": "Federated States of Micronesia",
                                               "Republic of Moldova": "Moldova",
                                               "Viet Nam": "Vietnam",
                                               "Venezuela (Bolivarian Republic of)": "Venezuela",
                                               "United States of America": "United States",
                                               "Türkiye": "Turkey",
                                               "United Republic of Tanzania": "Tanzania",
                                               "China, Taiwan Province of China": "Taiwan",
                                               "China, Macao SAR": "Macao"                                            
                                               }

country_name_replace_mapping_for_health_df = {"Congo": "Republic of the Congo",
                                              "Bolivia (Plurinational State of)": "Bolivia",
                                              "Iran (Islamic Republic of)": "Iran",
                                              "Lao People's Democratic Republic": "Laos",
                                              "Micronesia (Federated States of)": "Federated States of Micronesia",
                                              "Republic of Korea": "South Korea",
                                              "Republic of Moldova": "Moldova",
                                              "Syrian Arab Republic": "Syria",
                                              "The former Yugoslav Republic of Macedonia": "North Macedonia",
                                              "Türkiye": "Turkey",
                                              "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                                              "United Republic of Tanzania": "Tanzania",
                                              "United States of America": "United States",
                                              "Venezuela (Bolivarian Republic of)": "Venezuela",
                                              "Viet Nam": "Vietnam",
                                              "Côte d’Ivoire": "Côte d'Ivoire",
                                              "Gambia": "The Gambia"}


In [4]:
general_df['Country'] = general_df['Country'].replace(country_name_replace_mapping_for_general_df)

In [5]:
files = os.listdir("data/economy/")
for file in files:
    file_path = os.path.join("data/economy", file)
    indicator = file.split(".")[0]
    
    df = pd.read_csv(file_path)

    # Melt the DataFrame to transform the structure
    melted_df = df.melt(id_vars=df.columns[0], var_name='Year', value_name='Value')

    # Rename columns to match your desired structure
    melted_df.columns = ['Country', 'Year', indicator]

    # Sort by country and year if needed
    melted_df = melted_df.sort_values(by=['Country', 'Year'])

    # Convert the year column to integer type
    melted_df['Year'] = melted_df['Year'].astype(int)

    # Filter the year
    melted_df = melted_df[(melted_df['Year'] <= 2023) & (melted_df['Year'] >= 2000)]

    # Replace the country name for integration
    melted_df['Country'] = melted_df['Country'].replace(country_name_replace_mapping_for_economy_df)

    melted_df.reset_index(inplace=True, drop=True)
    
    general_df = pd.merge(general_df, melted_df, on=['Country', 'Year'])

In [6]:
general_df

,Country,location_code,country_code,Type,Year,total_population,male_population,female_population,population_density,life_expectancy_at_birth,male_life_expectancy_at_birth,female_life_expectancy_at_birth,infant_deaths_per_1000,mortality_between_15_60_per_1000,gdp,gdp_growth,gdp_per_capita,inflation_change
0,Afghanistan,4,AFG,Country/Area,2000,19542.982,9815.442,9727.541,30.099,55.298,53.762,56.858,90.813,322.256,NaN,NaN,NaN,NaN
1,Afghanistan,4,AFG,Country/Area,2001,19688.632,9895.467,9793.166,30.323,55.798,54.272,57.344,88.358,316.523,NaN,NaN,NaN,NaN
2,Afghanistan,4,AFG,Country/Area,2002,21000.256,10562.202,10438.055,32.343,56.454,55.227,57.672,85.779,306.104,4.367,NaN,233.433,5.1
3,Afghanistan,4,AFG,Country/Area,2003,22645.130,11397.483,11247.647,34.876,57.344,56.032,58.650,82.600,295.454,4.553,8.7,233.755,35.7
4,Afghanistan,4,AFG,Country/Area,2004,23553.551,11862.726,11690.825,36.276,57.944,56.658,59.216,79.936,288.209,5.146,0.7,254.259,16.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4651,Zimbabwe,716,ZWE,Country/Area,2019,15354.608,7231.989,8122.618,39.691,61.292,58.644,63.661,37.195,347.489,26.045,-6.3,1747.346,255.3
4652,Zimbabwe,716,ZWE,Country/Area,2020,15669.666,7385.220,8284.447,40.506,61.124,58.129,63.862,36.810,349.046,26.905,-7.8,1771.290,557.2
4653,Zimbabwe,716,ZWE,Country/Area,2021,15993.524,7543.690,8449.834,41.343,59.253,56.232,62.045,36.760,386.447,35.990,8.4,2323.090,98.5
4654,Zimbabwe,716,ZWE,Country/Area,2022,16320.537,7705.601,8614.935,42.188,59.391,56.428,62.094,37.421,382.439,31.490,6.2,1990.930,193.4


### Merge the health dataframe

In [8]:
df = pd.read_excel("data/health_data/water_services.xlsx")
df = df.iloc[1:]
df.head()

In [26]:
countries = df['Period']  # Add your list of countries here

# Creating an empty list to store the data
data = []

# Generating data for each country from 2000 to 2023
for country in countries:
    for year in range(2000, 2023):
        data.append({'Country': country, 'Year': year})

# Creating a DataFrame from the data list
new_df_water = pd.DataFrame(data)
new_df_water['Country'] = new_df_water['Country'].replace(country_name_replace_mapping_for_health_df)
new_df_water[['rural_safe_water_service', 'urban_safe_water_service', 'total_safe_water_service']] = df.iloc[:, :0:-1].values.reshape(195, 23, 3).reshape(-1, 3)

new_df_water.to_csv("data/health_data/water_services.csv", index=False)

In [22]:
df = pd.read_excel("data/health_data/sanitation_services.xlsx")
df = df.iloc[1:]
df.head()

,Period,2022,2022.1,2022.2,2021,2021.1,2021.2,2020,2020.1,2020.2,...,2003.2,2002,2002.1,2002.2,2001,2001.1,2001.2,2000,2000.1,2000.2
1,Afghanistan,55.95,70.39,50.71,54.3,68.67,49.17,52.65,66.95,47.62,...,21.29,22.54,32.31,19.75,20.98,30.76,18.2,20.97,30.76,18.2
2,Albania,99.3,99.31,99.28,99.3,99.31,99.28,99.3,99.31,99.28,...,86.43,90.61,97.02,85.67,90.03,96.97,84.91,89.49,96.93,84.16
3,Algeria,85.83,87.9,79.68,85.78,87.9,79.68,85.97,88.28,79.46,...,74.67,85.03,91.74,74.32,84.76,91.74,73.98,84.48,91.74,73.63
4,Andorra,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
5,Angola,52.18,65.31,24.16,51.92,65.31,24.16,51.66,65.31,24.16,...,10.3,30.42,49.49,9.38,28.99,48.5,8.45,27.56,47.51,7.53


In [27]:
countries = df['Period']  # Add your list of countries here

# Creating an empty list to store the data
data = []

# Generating data for each country from 2000 to 2023
for country in countries:
    for year in range(2000, 2023):
        data.append({'Country': country, 'Year': year})

# Creating a DataFrame from the data list
new_df_sanitation = pd.DataFrame(data)
new_df_sanitation['Country'] = new_df_sanitation['Country'].replace(country_name_replace_mapping_for_health_df)
new_df_sanitation[['rural_safe_sanitation_service', 'urban_safe_sanitation_service', 'total_safe_sanitation_service']] = df.iloc[:, :0:-1].values.reshape(195, 23, 3).reshape(-1, 3)

new_df_sanitation.to_csv("data/health_data/sanitation_services.csv", index=False)

In [7]:
new_df_water = pd.read_csv("data/health_data/water_services.csv")
new_df_sanitation = pd.read_csv("data/health_data/sanitation_services.csv")

health_df = pd.merge(new_df_water, new_df_sanitation, on=['Country', 'Year'])

general_df = pd.merge(general_df, health_df, on=['Country', 'Year'])

In [8]:

files = ["health_expenditure_over_gdp.csv", "health_expenditure_per_capita.csv"]
for file in files:
    file_path = os.path.join("data/health_data", file)
    indicator = file.split(".")[0]
    
    df = pd.read_csv(file_path)

    # Melt the DataFrame to transform the structure
    melted_df = df.melt(id_vars=df.columns[0], var_name='Year', value_name='Value')

    # Rename columns to match your desired structure
    melted_df.columns = ['Country', 'Year', indicator]

    # Sort by country and year if needed
    melted_df = melted_df.sort_values(by=['Country', 'Year'])

    # Convert the year column to integer type
    melted_df['Year'] = melted_df['Year'].astype(int)

    # Filter the year
    melted_df = melted_df[(melted_df['Year'] <= 2023) & (melted_df['Year'] >= 2000)]

    # Replace the country name for integration
    melted_df['Country'] = melted_df['Country'].replace(country_name_replace_mapping_for_health_df)
    
    melted_df[indicator] = melted_df[indicator].replace('[^\d.]', '', regex=True).astype(float)

    melted_df.reset_index(inplace=True, drop=True)
    
    general_df = pd.merge(general_df, melted_df, on=['Country', 'Year'])

### Region Country

In [9]:
df = pd.read_csv("data/region_country.csv")
df.sort_values("Country", inplace=True)
country_name_replace_mapping = {
    "Bolivia (Plurinational State of)": "Bolivia",
    "China, Hong Kong SAR": "Hong Kong",
    "China, Macao SAR": "Macao",
    "China, Taiwan Province of China": "Taiwan",
    "Micronesia (Fed. States of)": "Federated States of Micronesia",
    "Micronesia (Federated States of)": "Federated States of Micronesia",
    "Gambia": "The Gambia",
    "Iran (Islamic Republic of)": "Iran",
    "Lao People's Democratic Republic": "Laos",
    "Republic of Korea": "South Korea",
    "Republic of Moldova": "Moldova",
    "Congo": "Republic of the Congo",
    "Syrian Arab Republic": "Syria",
    "Türkiye": "Turkey",
    "United Republic of Tanzania": "Tanzania",
    "United States of America": "United States",
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "Viet Nam": "Vietnam",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "The former Yugoslav republic of Macedonia": "North Macedonia"
}
df['Country'] = df['Country'].replace(country_name_replace_mapping)
df.reset_index(inplace=True, drop=True)
df

,Region,Subregion,Country
0,ASIA,Southern Asia,Afghanistan
1,EUROPE,Southern Europe,Albania
2,AFRICA,Northern Africa,Algeria
3,OCEANIA,Polynesia,American Samoa
4,EUROPE,Southern Europe,Andorra
...,...,...,...
232,OCEANIA,Polynesia,Wallis and Futuna Islands
233,AFRICA,Northern Africa,Western Sahara
234,ASIA,Western Asia,Yemen
235,AFRICA,Eastern Africa,Zambia


In [58]:
df.to_csv("data/region_country.csv", index=False)

In [10]:
# Merge with general dataframe

general_df = pd.merge(general_df, df, on=['Country'])
general_df

,Country,location_code,country_code,Type,Year,total_population,male_population,female_population,population_density,life_expectancy_at_birth,...,rural_safe_water_service,urban_safe_water_service,total_safe_water_service,rural_safe_sanitation_service,urban_safe_sanitation_service,total_safe_sanitation_service,health_expenditure_over_gdp,health_expenditure_per_capita,Region,Subregion
0,Afghanistan,4,AFG,Country/Area,2000,19542.982,9815.442,9727.541,30.099,55.298,...,18.20,30.76,20.97,18.20,30.76,20.97,NaN,NaN,ASIA,Southern Asia
1,Afghanistan,4,AFG,Country/Area,2001,19688.632,9895.467,9793.166,30.323,55.798,...,18.20,30.76,20.98,18.20,30.76,20.98,NaN,NaN,ASIA,Southern Asia
2,Afghanistan,4,AFG,Country/Area,2002,21000.256,10562.202,10438.055,32.343,56.454,...,19.75,32.31,22.54,19.75,32.31,22.54,9.44,17.01,ASIA,Southern Asia
3,Afghanistan,4,AFG,Country/Area,2003,22645.130,11397.483,11247.647,34.876,57.344,...,21.29,33.85,24.10,21.29,33.85,24.10,8.94,17.81,ASIA,Southern Asia
4,Afghanistan,4,AFG,Country/Area,2004,23553.551,11862.726,11690.825,36.276,57.944,...,22.84,35.40,25.67,22.84,35.40,25.67,9.81,21.43,ASIA,Southern Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4109,Zimbabwe,716,ZWE,Country/Area,2017,14751.101,6940.631,7810.471,38.131,60.709,...,32.63,46.00,36.94,32.63,46.00,36.94,6.36,92.25,AFRICA,Eastern Africa
4110,Zimbabwe,716,ZWE,Country/Area,2018,15052.184,7086.002,7966.181,38.910,61.414,...,32.43,44.61,36.36,32.43,44.61,36.36,4.67,114.60,AFRICA,Eastern Africa
4111,Zimbabwe,716,ZWE,Country/Area,2019,15354.608,7231.989,8122.618,39.691,61.292,...,32.23,43.22,35.77,32.23,43.22,35.77,3.23,54.81,AFRICA,Eastern Africa
4112,Zimbabwe,716,ZWE,Country/Area,2020,15669.666,7385.220,8284.447,40.506,61.124,...,32.03,41.83,35.19,32.03,41.83,35.19,2.95,50.68,AFRICA,Eastern Africa


In [11]:
general_df['Subregion'].unique()

array(['Southern Asia', 'Southern Europe', 'Northern Africa',
       'Middle Africa', 'Caribbean', 'South America', 'Western Asia',
       'Australia/New Zealand', 'Western Europe', 'Eastern Europe',
       'Central America', 'Western Africa', 'Southern Africa',
       'South-Eastern Asia', 'Eastern Africa', 'Eastern Asia',
       'Northern Europe', 'Melanesia', 'Central Asia', 'Micronesia',
       'Polynesia'], dtype=object)

### Develop status

In [12]:
develop_df = pd.read_csv("data/Life_Expectancy_Data.csv")[['Country', 'Status']]
develop_df.drop_duplicates(keep='first', inplace=True)
develop_df['Country'] = develop_df['Country'].replace(country_name_replace_mapping)
develop_df

,Country,Status
0,Afghanistan,Developing
16,Albania,Developing
32,Algeria,Developing
48,Angola,Developing
64,Antigua and Barbuda,Developing
...,...,...
2858,Venezuela,Developing
2874,Vietnam,Developing
2890,Yemen,Developing
2906,Zambia,Developing


In [62]:
general_df = pd.merge(general_df, develop_df, on='Country')

In [63]:
general_df.to_csv("data/integrated/integrated_data.csv", index=False)

In [48]:
general_df['Country'].unique().shape

(185,)